# Detrend Data<a id='top' class="tocSkip"> </a>

This tutorial shows how to detrend data

It is a commanly used technic to *detrend* data before doing further diagnosis, this tutorials shows how to do so using [CDAT](https://cdat.llnl.gov)


The CDAT software was developed by LLNL. This tutorial was written by Charles Doutriaux. This work was performed under the auspices of the U.S. Department of Energy by Lawrence Livermore National Laboratory under Contract DE-AC52-07NA27344.

[Download the Jupyter Notebook](Detrend_data.ipynb)



<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prepare-Notebook-and-Data" data-toc-modified-id="Prepare-Notebook-and-Data-1">Prepare Notebook and Data</a></span></li><li><span><a href="#Removing-Annual-Cycle" data-toc-modified-id="Removing-Annual-Cycle-2">Removing Annual Cycle</a></span></li><li><span><a href="#Detrend-Data" data-toc-modified-id="Detrend-Data-3">Detrend Data</a></span></li></ul></div>

## Prepare Notebook and Data
[Back to Top](#top)

In [ ]:
from __future__ import print_function
import cdms2
import MV2
import genutil
import cdutil
import vcs
import os
import requests

# This is a subset over North America
filename = 'tas_Amon_IPSL-CM5A-LR_1pctCO2_r1i1p1_185001-198912.nc'
if not os.path.exists(filename):
    r = requests.get("https://cdat.llnl.gov/cdat/sample_data/notebooks/{}".format(filename), stream=True)
    with open(filename,"wb") as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:  # filter local_filename keep-alive new chunks
                f.write(chunk)


f = cdms2.open(filename)

data = f("tas")

Now to demonstrate `genutil` capabilities and highlight some scientific points about operations order, let's mask some data and compute the time serie

In [ ]:
# Mask data where greater than maximum less 2 K
data = MV2.masked_greater(data, data.max()-2)

# Time series
data_ts = genutil.averager(data, axis='xy')

Let's take a look at time serie and note the trend and annual cycle

In [ ]:
x = vcs.init(bg=True, geometry=(800,600))
line = vcs.create1d()
line.markersize = .5
x.plot(data_ts, line)

## Removing Annual Cycle 
[Back to Top](#top)

First we will remove the annual cycle for these **monthly** data. Because it is masked order matter as we will demo.

In [ ]:
data_departures = cdutil.times.ANNUALCYCLE.departures(data)
data_departures_ts = genutil.averager(data_departures, axis='xy')
data_ts_departures = cdutil.times.ANNUALCYCLE.departures(data_ts)

Notice how the trends shows up much nicer now!

In [ ]:
x.clear()
x.plot(data_departures_ts)

Note please note the importance of order operation when missing data is present both time series are slightly different and the later years (where missing data occurs)

In [ ]:
x.clear()
x.plot(data_departures_ts - data_ts_departures)

## Detrend Data
[Back to Top](#top)

First we will compute the trend, other `time` note that the index of time can be anything, `genutil` will determine its index.

After computation we lose the time axis, also note the units, based on units on time axis, since units are in `days since XXX` the coeef/trend is in `K/day`

In [ ]:
coeff, intercept = genutil.statistics.linearregression(data_departures, axis="t")
print("Shapes: coeff {}, intercept {}".format(coeff.shape, intercept.shape))
# Let's do the same for the time serie
coeff_ts, intercept_ts = genutil.statistics.linearregression(data_departures_ts, axis="t")

Now let's actually remove the trend
We will need

In [ ]:
times = MV2.array(data.getTime()[:])
times.setAxis(0, data.getTime())

# since time is not necessarily on index 0 we need to use the grower function
# we use data as first argument to ensure same order
tmp, full_times = genutil.grower(data_departures, times)
# same for cefficient
tmp, coeff = genutil.grower(data, coeff)
# same for intercept
tmp, intercept = genutil.grower(data, intercept)

data_departures_detrend = data_departures - full_times * coeff - intercept
data_departures_ts_detrend = data_departures_ts - times * coeff_ts - intercept_ts
x.clear()
x.plot(data_departures_ts_detrend)

Here again the order matters

In [ ]:
data_departures_detrend_ts = genutil.averager(data_departures_detrend, axis='xy')

x.clear()
x.plot(data_departures_detrend_ts - data_departures_ts_detrend)